In [1]:
from __future__ import print_function, division
%matplotlib inline


import torch
import os
from torchvision import transforms, datasets, models

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
from tqdm import tqdm_notebook as tqdm
import torchvision
import matplotlib.pyplot as plt
import time
import copy
from torch.utils.data.sampler import SubsetRandomSampler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model_ft = models.resnet50(pretrained = True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 6)
model_ft = model_ft.to(device)

In [5]:
torch.load('smartbin_resnet50_00.pth')

RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1535491974311/work/aten/src/THC/THCGeneral.cpp:663